In [11]:
from importlib import reload
import torch
import numpy as np
import tempfile
import gym
import os

from spacelib.dataset import RecurrentReplayBuffer


In [12]:
# import roboschool
# env = gym.make('RoboschoolHalfCheetah-v1')
# observation_space = env.observation_space
# action_space = env.action_space

import minerl
minerl_env_name = 'MineRLObtainDiamondDense-v0'
observation_space = gym.envs.registration.spec(minerl_env_name)._kwargs['observation_space']
action_space = gym.envs.registration.spec(minerl_env_name)._kwargs['action_space']

In [28]:
n_episodes = 3
workdir = tempfile.mkdtemp(prefix='spacelib')
print(f"Saving files in {workdir}")

replay_buffer = RecurrentReplayBuffer(observation_space, action_space, data_root=workdir)

for n_ep in range(n_episodes):

#     obs = env.reset()
    obs = observation_space.sample()
    
    done = False
    replay_buffer.begin_episode()
    episode_length = 0
    
    while not done:

        act = action_space.sample()
        
#         new_obs, rew, done, _ = env.step(act)
        new_obs = observation_space.sample()
        rew = episode_length
        done = episode_length > 100 and np.random.randn()<-3
        
        replay_buffer.append((obs, act, rew, done))
        
        obs = new_obs
        episode_length += 1
        
    replay_buffer.end_episode()
    
for path, dirs, files in os.walk(workdir):
    if any(f.endswith('.dat') for f in files):
        print('\n',path)
        for datfile in files:
            size_kb = os.path.getsize(os.path.join(path, datfile))//1024
            print(f"  .../{datfile}: {size_kb} KB")

Saving files in /tmp/spacelibevxyuyd4

 /tmp/spacelibevxyuyd4/lumh20aDjh
  .../1.0.0.dat: 2916 KB
  .../0.0.0.dat: 75 KB

 /tmp/spacelibevxyuyd4/JikHOX9e9c
  .../1.0.0.dat: 15108 KB
  .../0.0.0.dat: 393 KB

 /tmp/spacelibevxyuyd4/T7NZ5NNibR
  .../1.0.0.dat: 1584 KB
  .../0.0.0.dat: 41 KB


In [29]:
%%timeit -n 10
replay_buffer.sample_sequence(length=64, batch_size=32, device=torch.device('cuda'))

35.2 ms ± 852 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit -n 10
replay_buffer.sample_sequence(length=256, batch_size=8, device=torch.device('cuda'))

30.9 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
from spacelib.flatter import Flatter
sample_sequence_batch = replay_buffer.sample_sequence(length = 64, batch_size=32, device=torch.device('cuda'))
Flatter(observation_space).unflatten(sample_sequence_batch.obs)['pov'].shape

torch.Size([32, 64, 64, 64, 3])

In [35]:
# In the above made-up example, the reward increments at each time step. 
# As a simple sanity check, we can verify that this holds in sampled sequences.
sample_sequence_batch.rew

tensor([[   4.,    5.,    6.,  ...,   65.,   66.,   67.],
        [ 149.,  150.,  151.,  ...,  210.,  211.,  212.],
        [   7.,    8.,    9.,  ...,   68.,   69.,   70.],
        ...,
        [ 355.,  356.,  357.,  ...,  416.,  417.,  418.],
        [ 790.,  791.,  792.,  ...,  851.,  852.,  853.],
        [1191., 1192., 1193.,  ..., 1252., 1253., 1254.]], device='cuda:0')